In [1]:
# Load the embeddings
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Load our own Word2Vec embedding
own_w2v = KeyedVectors.load_word2vec_format("own_word2vec.model", binary=True)

In [3]:
# Load pre-trained Word2Vec embedding
google_w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [4]:
glove_file = "glove.6B.300d.txt"
glove_embeddings = dict()

with open(glove_file, "r", encoding = "utf-8") as file:
    for line in tqdm(file, desc="Loading embeddings"):
        values = line.split()
        word = values[0]
        try:
            vector = np.asarray(values[1:], dtype=np.float32)
            glove_embeddings[word] = vector
        except ValueError:
            print("Could not convert values to float: ")
        glove_embeddings[word] = vector
        
glove_w2v = Word2Vec(
    sentences=None,  # Don't train on any text data
    vector_size=300,  # Set the embedding dimension
    min_count=1,  # Include all words in the dictionary
)
glove_w2v.wv.add_vectors(list(glove_embeddings.keys()), list(glove_embeddings.values()))

Loading embeddings: 400000it [00:24, 16637.98it/s]


In [5]:
# Load the dataset (e.g., IMDB movie reviews)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)

In [6]:
# Convert the word IDs to text
word_index = tf.keras.datasets.imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
x_train_text = [' '.join([reverse_word_index.get(i - 3, '?') for i in x]) for x in x_train]
x_test_text = [' '.join([reverse_word_index.get(i - 3, '?') for i in x]) for x in x_test]

In [7]:
# Tokenize the text and pad the sequences
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train_text)
x_train_seq = tokenizer.texts_to_sequences(x_train_text)
x_test_seq = tokenizer.texts_to_sequences(x_test_text)
x_train_pad = pad_sequences(x_train_seq, maxlen=100)
x_test_pad = pad_sequences(x_test_seq, maxlen=100)

In [8]:
# Define own word2vec embedding layers
pretrained_own_emb_layer = Embedding(input_dim = own_w2v.vectors.shape[0], output_dim = 300, input_length = 100,
                                     weights = [own_w2v.vectors], trainable = False)
# Define google word2vec embedding layers
pretrained_google_emb_layer = Embedding(input_dim = google_w2v.vectors.shape[0], output_dim = 300, input_length = 100,
                                     weights = [google_w2v.vectors], trainable = False)
# Define glove word2vec embedding layers
pretrained_glove_emb_layer = Embedding(input_dim = glove_w2v.wv.vectors.shape[0], output_dim = 300, input_length = 100,
                                       weights = [glove_w2v.wv.vectors], trainable = False)

### Own Word2vec Model

In [9]:
# Define the neural network architecture
input_layer = Input(shape=(100,))
emb_layer = pretrained_own_emb_layer(input_layer)
lstm_layer = LSTM(units=64, dropout=0.2, recurrent_dropout=0.2)(emb_layer)
dropout_layer = Dropout(rate=0.2)(lstm_layer)
output_layer = Dense(units=1, activation='sigmoid')(dropout_layer)

In [10]:
# Define the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 300)          4369500   
                                                                 
 lstm (LSTM)                 (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 4,463,005
Trainable params: 93,505
Non-trainable params: 4,369,500
_________________________________________________________________
None


In [11]:
# Train the model
model.fit(x_train_pad, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 35s 211ms/step - loss: 0.6703 - accuracy: 0.5760 - val_loss: 0.6432 - val_accuracy: 0.6178
Epoch 2/10
157/157 [==============================] - 38s 243ms/step - loss: 0.6172 - accuracy: 0.6564 - val_loss: 0.6054 - val_accuracy: 0.6662
Epoch 3/10
157/157 [==============================] - 47s 298ms/step - loss: 0.5983 - accuracy: 0.6750 - val_loss: 0.6317 - val_accuracy: 0.6370
Epoch 4/10
157/157 [==============================] - 48s 306ms/step - loss: 0.5635 - accuracy: 0.7066 - val_loss: 0.5543 - val_accuracy: 0.7170
Epoch 5/10
157/157 [==============================] - 51s 326ms/step - loss: 0.5415 - accuracy: 0.7198 - val_loss: 0.4960 - val_accuracy: 0.7558
Epoch 6/10
157/157 [==============================] - 49s 315ms/step - loss: 0.5086 - accuracy: 0.7480 - val_loss: 0.4758 - val_accuracy: 0.7712
Epoch 7/10
157/157 [==============================] - 51s 328ms/step - loss: 0.4763 - accuracy: 0.7704 - val_loss: 0.4583 - val_ac

In [12]:
# Evaluate the model
y_pred = model.predict(x_test_pad) > 0.5
acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

782/782 [==============================] - 35s 45ms/step
Accuracy: 0.79084


### Google Word2vec Model

In [13]:
# Define the neural network architecture
input_layer = Input(shape=(100,))
emb_layer = pretrained_google_emb_layer(input_layer)
lstm_layer = LSTM(units=64, dropout=0.2, recurrent_dropout=0.2)(emb_layer)
dropout_layer = Dropout(rate=0.2)(lstm_layer)
output_layer = Dense(units=1, activation='sigmoid')(dropout_layer)

In [14]:
# Define the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 100, 300)          900000000 
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 900,093,505
Trainable params: 93,505
Non-trainable params: 900,000,000
_________________________________________________________________
None


In [15]:
# Train the model
model.fit(x_train_pad, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 44s 267ms/step - loss: 0.6727 - accuracy: 0.5762 - val_loss: 0.6444 - val_accuracy: 0.6180
Epoch 2/10
157/157 [==============================] - 46s 296ms/step - loss: 0.6202 - accuracy: 0.6519 - val_loss: 0.5811 - val_accuracy: 0.6976
Epoch 3/10
157/157 [==============================] - 48s 305ms/step - loss: 0.5868 - accuracy: 0.6887 - val_loss: 0.5703 - val_accuracy: 0.6982
Epoch 4/10
157/157 [==============================] - 49s 311ms/step - loss: 0.5584 - accuracy: 0.7117 - val_loss: 0.5380 - val_accuracy: 0.7278
Epoch 5/10
157/157 [==============================] - 48s 305ms/step - loss: 0.5356 - accuracy: 0.7268 - val_loss: 0.5175 - val_accuracy: 0.7416
Epoch 6/10
157/157 [==============================] - 49s 310ms/step - loss: 0.5211 - accuracy: 0.7383 - val_loss: 0.5811 - val_accuracy: 0.7096
Epoch 7/10
157/157 [==============================] - 49s 312ms/step - loss: 0.5056 - accuracy: 0.7489 - val_loss: 0.4779 - val_ac

In [16]:
# Evaluate the model
y_pred = model.predict(x_test_pad) > 0.5
acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

782/782 [==============================] - 39s 49ms/step
Accuracy: 0.79248


### GloVe Word2vec Model

In [17]:
# Define the neural network architecture
input_layer = Input(shape=(100,))
emb_layer = pretrained_glove_emb_layer(input_layer)
lstm_layer = LSTM(units=64, dropout=0.2, recurrent_dropout=0.2)(emb_layer)
dropout_layer = Dropout(rate=0.2)(lstm_layer)
output_layer = Dense(units=1, activation='sigmoid')(dropout_layer)

In [18]:
# Define the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 100, 300)          120000000 
                                                                 
 lstm_2 (LSTM)               (None, 64)                93440     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 120,093,505
Trainable params: 93,505
Non-trainable params: 120,000,000
_________________________________________________________________
None


In [19]:
# Train the model
model.fit(x_train_pad, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 47s 290ms/step - loss: 0.6513 - accuracy: 0.6098 - val_loss: 0.5942 - val_accuracy: 0.6904
Epoch 2/10
157/157 [==============================] - 46s 295ms/step - loss: 0.5972 - accuracy: 0.6775 - val_loss: 0.6107 - val_accuracy: 0.6768
Epoch 3/10
157/157 [==============================] - 48s 308ms/step - loss: 0.5489 - accuracy: 0.7178 - val_loss: 0.5476 - val_accuracy: 0.7246
Epoch 4/10
157/157 [==============================] - 49s 310ms/step - loss: 0.5039 - accuracy: 0.7497 - val_loss: 0.5119 - val_accuracy: 0.7546
Epoch 5/10
157/157 [==============================] - 52s 333ms/step - loss: 0.4663 - accuracy: 0.7764 - val_loss: 0.4748 - val_accuracy: 0.7708
Epoch 6/10
157/157 [==============================] - 50s 321ms/step - loss: 0.4285 - accuracy: 0.8001 - val_loss: 0.4899 - val_accuracy: 0.7598
Epoch 7/10
157/157 [==============================] - 52s 333ms/step - loss: 0.4116 - accuracy: 0.8055 - val_loss: 0.4275 - val_ac

In [20]:
# Evaluate the model
y_pred = model.predict(x_test_pad) > 0.5
acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

782/782 [==============================] - 52s 67ms/step
Accuracy: 0.81812
